In [1]:
%reload_ext autoreload
%autoreload 2

import polars as pl
from statsforecast import StatsForecast
from statsforecast.models import SeasonalNaive

from polars_ts.models.scum import SCUM

# Create sample dataframe with columns `unique_id`, `ds`, and `y`.
df = (
    pl.scan_parquet("https://datasets-nixtla.s3.amazonaws.com/m4-hourly.parquet")
    .filter(pl.col("unique_id").is_in(["H1", "H2", "H3"]))
    .collect()
)

season_length = 24

# Now, build an ensemble of SCUM + other models
models = [
    SCUM(season_length=season_length),
    SeasonalNaive(season_length=season_length),
]

sf = StatsForecast(models=models, freq=1, n_jobs=-1, verbose=True)

# Fit the models on your data
sf.fit(df=df)

# Forecast 5 steps ahead
fc = sf.predict(5, level=[95])
display(fc)

unique_id,ds,SCUM,SCUM-lo-95,SCUM-hi-95,SeasonalNaive,SeasonalNaive-lo-95,SeasonalNaive-hi-95
str,i64,f64,f64,f64,f64,f64,f64
"""H1""",749,598.242667,561.300875,636.38217,635.0,518.683456,751.316544
"""H1""",750,535.115589,491.823268,578.080455,572.0,455.683456,688.316544
"""H1""",751,495.312065,446.505672,543.455378,532.0,415.683456,648.316544
"""H1""",752,463.032106,410.923097,514.41964,493.0,376.683456,609.316544
"""H1""",753,443.774539,388.701325,498.527075,477.0,360.683456,593.316544
…,…,…,…,…,…,…,…
"""H3""",749,1634.270533,1561.98337,1708.928776,1585.0,1369.519464,1800.480536
"""H3""",750,1582.385899,1482.424673,1681.362417,1473.0,1257.519464,1688.480536
"""H3""",751,1479.339278,1358.629162,1598.416073,1317.0,1101.519464,1532.480536
